<a href="https://colab.research.google.com/github/helenawsu/trip-planner/blob/main/trip_planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys

if 'google.colab' in sys.modules:
    %pip install pyomo >/dev/null 2>/dev/null
    %pip install highspy >/dev/null 2>/dev/null

solver = 'appsi_highs'

import pyomo.environ as pyo
SOLVER = pyo.SolverFactory(solver)

assert SOLVER.available(), f"Solver {solver} is not available."

In [65]:
from pyomo.environ import *
import math

#----------------------------------
# Event class definition
#----------------------------------
class Event:
    def __init__(self, name, time_cost, money_cost, pref_point, etype, energy_cost=0, location=None):
        self.name = name
        self.time_cost    = math.ceil(time_cost)
        self.money_cost   = money_cost
        self.pref_point   = pref_point
        self.etype        = etype
        self.energy_cost  = energy_cost   # drains (+) or restores (–)
        self.location     = location      # (lat, lon) or None

#----------------------------------
# Define all events
#----------------------------------
events = [
    Event('drive_out', 4.5, 0,   0,   'transportation', energy_cost=2, location=None),
    Event('drive_in',  4.5, 0,   0,   'transportation', energy_cost=2, location=None),

    Event('boating',           2,   0, 8, 'activity', energy_cost=3, location=(40.4977, -121.4207)),
    Event('hydrothermal',      2,   0, 5, 'activity', energy_cost=3, location=(40.4977, -121.4207)),
    Event('peak',              5,   0, 8, 'activity', energy_cost=5, location=(40.4977, -121.4207)),
    Event('stargazing',        2,   0, 5, 'activity', energy_cost=2, location=(40.4977, -121.4207)),
    Event('upper_bidwell',     3,   0, 5, 'activity', energy_cost=3, location=(39.7803, -121.7480)),
    Event('north_table_mountain', 3, 0, 5, 'activity', energy_cost=3, location=(39.7803, -121.7480)),

    Event('lodging1_campcabin',   8,   0, 0, 'lodging', energy_cost=-5, location=(40.4977, -121.4207)),
    Event('lodging2_nearlassen',   8, 200, 0, 'lodging', energy_cost=-5, location=(40.2564, -121.1453)),
    Event('lodging3_ontheway',     8, 150, 0, 'lodging', energy_cost=-5, location=(39.7803, -121.7480)),

    Event('rest',             1,   0, 0, 'rest',   energy_cost=-2, location=None),
    Event('free',             1,   0, 0, 'free',   energy_cost= 0, location=None),
]

#----------------------------------
# Build Pyomo model
#----------------------------------
model = ConcreteModel()
model.H = RangeSet(0, 63)
model.A = Set(initialize=[e.name for e in events])

# Parameter dicts
time_cost    = {e.name: e.time_cost    for e in events}
cost         = {e.name: e.money_cost   for e in events}
pref         = {e.name: e.pref_point   for e in events}
etype        = {e.name: e.etype        for e in events}
energy_total = {e.name: e.energy_cost  for e in events}

model.duration       = Param(model.A, initialize=time_cost)
model.cost           = Param(model.A, initialize=cost)
model.pref           = Param(model.A, initialize=pref)
model.budget         = Param(initialize=2000)
model.energy_total   = Param(model.A, initialize=energy_total)
model.initial_energy = Param(initialize=5)

# Build event‐map & distance lookup
from math import sqrt as _sqrt
ev_map = {e.name: e for e in events}
dist_data = {}
for a in model.A:
    for b in model.A:
        la, lb = ev_map[a].location, ev_map[b].location
        if la and lb:
            dx, dy = la[0]-lb[0], la[1]-lb[1]
            dist_data[(a,b)] = _sqrt(dx*dx + dy*dy)
        else:
            dist_data[(a,b)] = 0.0

model.dist       = Param(model.A, model.A, initialize=dist_data)
model.loc_weight = Param(initialize=10.0)

#----------------------------------
# Decision variables
#----------------------------------
model.x     = Var(model.H, model.A, domain=Binary)
model.y     = Var(model.A, domain=NonNegativeIntegers)   # # of blocks
model.start = Var(model.A, model.H, domain=Binary)
model.E     = Var(model.H, domain=NonNegativeReals)

#----------------------------------
# Original constraints…
#----------------------------------
def event_once(m,a):
    return m.y[a] <= 1 if etype[a] in ('activity','transportation') else Constraint.Skip
model.EventOnce = Constraint(model.A, rule=event_once)

model.DurationLink = Constraint(model.A,
    rule=lambda m,a: sum(m.x[h,a] for h in m.H) == m.duration[a]*m.y[a])
model.OneHour      = Constraint(model.H,
    rule=lambda m,h: sum(m.x[h,a] for a in m.A)==1)
model.TotalTime    = Constraint(expr=sum(model.duration[a]*model.y[a] for a in model.A)<=64)
model.Budget       = Constraint(expr=sum(model.cost[a]*model.y[a]     for a in model.A)<=model.budget)

model.StartLink    = Constraint(model.A,
    rule=lambda m,a: sum(m.start[a,h] for h in m.H if h+m.duration[a]<=64)==m.y[a])
model.XStart       = Constraint(model.A, model.H,
    rule=lambda m,a,h: m.x[h,a]==sum(m.start[a,s] for s in m.H if s<=h<s+m.duration[a]))

model.ActivityOnce    = Constraint(model.A,
    rule=lambda m,a: (m.y[a]<=1) if etype[a]=='activity'      else Constraint.Skip)
model.TransportOnce   = Constraint(model.A,
    rule=lambda m,a: (m.y[a]<=1) if etype[a]=='transportation' else Constraint.Skip)

model.StartDriveOut   = Constraint(expr=model.start['drive_out',0]==1)
last_in = 64 - math.ceil(events[1].time_cost)
model.StartDriveIn    = Constraint(expr=model.start['drive_in',last_in]==1)

model.ParkSeq = Constraint(model.H, model.A,
    rule=lambda m,h,a: (m.x[h,a]==0) if etype[a]=='activity' and h<math.ceil(events[0].time_cost) else Constraint.Skip)

lodges = [e.name for e in events if e.etype=='lodging']
model.LodgingTime = Constraint(model.H, model.A,
    rule=lambda m,h,a: (m.x[h,a]==0) if a in lodges and not((16<=h<=24) or (40<=h<=48)) else Constraint.Skip)
model.LodgeCount  = Constraint(expr=sum(model.y[l] for l in lodges)==2)

model.Stargaze = Constraint(model.H,
    rule=lambda m,h: (m.x[h,'stargazing']==0)
        if not(((8+h)%24)>=21 or ((8+h)%24)<5) else Constraint.Skip)

def energy_balance(m,h):
    change = sum(m.energy_total[a]*m.start[a,h-m.duration[a]+1]
                 for a in m.A if 0<=h-m.duration[a]+1<=59)
    return m.E[h] == (m.initial_energy - change if h==0 else m.E[h-1] - change)
model.EnergyBal = Constraint(model.H, rule=energy_balance)
model.EnergyPos = Constraint(model.H, rule=lambda m,h: m.E[h]>=0)
model.EnergyCap = Constraint(model.H, rule=lambda m,h: m.E[h]<=m.initial_energy)

#----------------------------------
# Compact “real‐event” sequence
#----------------------------------
# 1) R = events with a location
R = [a for a in model.A if ev_map[a].location is not None]
model.R = Set(initialize=R)

# 2) K = max # real events you could ever pick
K = len(R)
model.K = RangeSet(1, K)

# 3) u[a,k] = 1 iff real event a is placed in position k
model.u = Var(model.R, model.K, domain=Binary)

# 4) each event a with y[a]=1 gets exactly one slot
def one_pos_per_event(m,a):
    return sum(m.u[a,k] for k in m.K) == m.y[a]
model.OnePosPerEvent = Constraint(model.R, rule=one_pos_per_event)

# 5) each slot k holds at most one event
def at_most_one_event_per_pos(m,k):
    return sum(m.u[a,k] for a in m.R) <= 1
model.AtMostOnePerPos = Constraint(model.K, rule=at_most_one_event_per_pos)

# 6) (optional) no gaps: slot k must be used before k+1
def no_gaps(m,k):
    if k < K:
        return sum(m.u[a,k]   for a in m.R) >= sum(m.u[a,k+1] for a in m.R)
    return Constraint.Skip
model.NoGaps = Constraint(model.K, rule=no_gaps)

# 7) link consecutive slots via v[a,b,k]
model.v = Var(model.R, model.R, model.K, domain=Binary)
model.V1 = Constraint(model.R, model.R, model.K,
    rule=lambda m,a,b,k: m.v[a,b,k] <= m.u[a,k])
model.V2 = Constraint(model.R, model.R, model.K,
    rule=lambda m,a,b,k: m.v[a,b,k] <= m.u[b,k+1] if k < K else Constraint.Skip)
model.V3 = Constraint(model.R, model.R, model.K,
    rule=lambda m,a,b,k: m.v[a,b,k] >= m.u[a,k] + (m.u[b,k+1] if k < K else 0) - 1)
# 8) Define actual start‐time variable for each event
model.T = Var(model.A, domain=NonNegativeReals)

def Tdef(m,a):
    return m.T[a] == sum(h * m.start[a,h] for h in m.H)
model.Tdef = Constraint(model.A, rule=Tdef)

# 9) Tie compact slots (u) to chronological order
M = 64
def chrono(m, a, b, k):
    if k < K:
        # if u[a,k]=u[b,k+1]=1, then T[a] <= T[b]
        return m.T[a] <= m.T[b] + M*(2 - m.u[a,k] - m.u[b,k+1])
    return Constraint.Skip

model.ChronoOrder = Constraint(model.R, model.R, model.K, rule=chrono)


#----------------------------------
# Objective
#----------------------------------
pref_term = sum(model.pref[a] * model.y[a] for a in model.A)

dist_term = sum(
    dist_data[(a,b)] * model.v[a,b,k]
    for a in model.R for b in model.R for k in model.K if k < K
)

model.obj = Objective(
    expr = pref_term - model.loc_weight * dist_term,
    sense = maximize
)


In [66]:
# Solve & Print (including repeated real events)
def solve_and_print():
    res = SOLVER.solve(model)
    if res.solver.termination_condition != TerminationCondition.optimal:
        print('No optimal solution found.')
        return

    # 1) Total preference
    total_pref = sum(model.pref[a] * model.y[a].value for a in model.A)
    print(f"Total preference points: {total_pref}")

    # 2) Build hourly blocks (unchanged)
    sched = [(h, a) for h in model.H for a in model.A if model.x[h, a].value > 0.5]
    sched.sort()
    blocks = []
    if sched:
        s, act = sched[0]; prev = s
        for h, a in sched[1:]:
            if a == act and h == prev + 1:
                prev = h
            else:
                blocks.append((s, prev, act))
                s = prev = h
                act = a
        blocks.append((s, prev, act))

    # 3) Extract real events in true chronological order (allow repeats)
    real_seq = [(h, a) for (h, a) in sched if a in model.R]

    print("\nChronological real-event ordering (hour → event):")
    for h, a in real_seq:
        print(f"  {h:02d}:00 → {a}")

    # 4) Compute raw distance by walking that list
    raw_distance = 0.0
    for (_, a1), (_, a2) in zip(real_seq, real_seq[1:]):
        raw_distance += dist_data[(a1, a2)]
    weighted_distance = model.loc_weight.value * raw_distance

    print(f"\nTotal travel distance (real-event hops): {raw_distance:.2f}")
    print(f"Weighted distance penalty:       {weighted_distance:.2f}\n")

    # 5) Print the full schedule with energy
    for s, e, a in blocks:
        sd, sh = divmod(8 + s, 24)
        ed, eh = divmod(8 + e + 1, 24)
        interval = f"Day {sd+1} {sh:02d}:00–Day {ed+1} {eh:02d}:00"
        energy_before = (model.initial_energy.value if s == 0
                         else model.E[s-1].value)
        energy_after  = model.E[e].value
        print(f"{interval}: {a} "
              f"(Energy before: {energy_before:.2f}, after: {energy_after:.2f})")

# Run it
solve_and_print()


Total preference points: 35.999999999999986

Chronological real-event ordering (hour → event):
  09:00 → boating
  10:00 → boating
  14:00 → stargazing
  15:00 → stargazing
  16:00 → lodging1_campcabin
  17:00 → lodging1_campcabin
  18:00 → lodging1_campcabin
  19:00 → lodging1_campcabin
  20:00 → lodging1_campcabin
  21:00 → lodging1_campcabin
  22:00 → lodging1_campcabin
  23:00 → lodging1_campcabin
  24:00 → peak
  25:00 → peak
  26:00 → peak
  27:00 → peak
  28:00 → peak
  32:00 → hydrothermal
  33:00 → hydrothermal
  35:00 → north_table_mountain
  36:00 → north_table_mountain
  37:00 → north_table_mountain
  40:00 → lodging3_ontheway
  41:00 → lodging3_ontheway
  42:00 → lodging3_ontheway
  43:00 → lodging3_ontheway
  44:00 → lodging3_ontheway
  45:00 → lodging3_ontheway
  46:00 → lodging3_ontheway
  47:00 → lodging3_ontheway
  48:00 → upper_bidwell
  49:00 → upper_bidwell
  50:00 → upper_bidwell

Total travel distance (real-event hops): 0.79
Weighted distance penalty:       7.89


In [36]:
dist_data

{('drive_out', 'drive_out'): 0.0,
 ('drive_out', 'drive_in'): 0.0,
 ('drive_out', 'boating'): 0.0,
 ('drive_out', 'hydrothermal'): 0.0,
 ('drive_out', 'peak'): 0.0,
 ('drive_out', 'stargazing'): 0.0,
 ('drive_out', 'upper_bidwell'): 0.0,
 ('drive_out', 'north_table_mountain'): 0.0,
 ('drive_out', 'lodging1_campcabin'): 0.0,
 ('drive_out', 'lodging2_nearlassen'): 0.0,
 ('drive_out', 'lodging3_ontheway'): 0.0,
 ('drive_out', 'rest'): 0.0,
 ('drive_in', 'drive_out'): 0.0,
 ('drive_in', 'drive_in'): 0.0,
 ('drive_in', 'boating'): 0.0,
 ('drive_in', 'hydrothermal'): 0.0,
 ('drive_in', 'peak'): 0.0,
 ('drive_in', 'stargazing'): 0.0,
 ('drive_in', 'upper_bidwell'): 0.0,
 ('drive_in', 'north_table_mountain'): 0.0,
 ('drive_in', 'lodging1_campcabin'): 0.0,
 ('drive_in', 'lodging2_nearlassen'): 0.0,
 ('drive_in', 'lodging3_ontheway'): 0.0,
 ('drive_in', 'rest'): 0.0,
 ('boating', 'drive_out'): 0.0,
 ('boating', 'drive_in'): 0.0,
 ('boating', 'boating'): 0.0,
 ('boating', 'hydrothermal'): 0.0,
 (